# TiTiler Tests

In [29]:
from maap_test_helper import *
import requests
from PIL import Image
from io import BytesIO
from cogeo_mosaic.mosaic import MosaicJSON


In [30]:
# papermill parameters cell

stage = "dit"


In [31]:
maap_host=populate_maap_api_host(stage)

titiler_host = maap_host.replace("api.", "titiler.")
titiler_host = titiler_host.replace("ops.", "")

cog_url = "s3://nasa-maap-data-store/file-staging/nasa-map/SRTMGL1_COD___001/N48E006.SRTMGL1.tif"


In [32]:

url = f"https://{titiler_host}/cog/tiles/WebMercatorQuad/0/0/0@1x.png"

res = requests.get(url, params={"TileMatrixSetId": "WebMercatorQuad", "url": cog_url})

assert res.status_code == 200
assert Image.open(BytesIO(res.content)).format == "PNG"

In [33]:
url = f"https://{titiler_host}/cog/bounds"
res = requests.get(url, params={"url": cog_url})

assert res.status_code == 200
assert res.json()['bounds'] == [5.9998611111111115,
                                47.999861111111116, 7.000138888888889, 49.00013888888889]


In [38]:
r_stac = requests.post(
    "https://cmr-stac.dit.maap-project.org/stac/NASA_MAAP/search",
    headers={
      "Content-Type": "application/json",
      "Accept": "application/geo+json, application/json",
    }, 
    json={
      "collections": ["SRTMGL1_COD.v001"],
      "bbox": "4,42,16,48",
      "limit": 120
    }
)

items = r_stac.json()

mosaicdata = MosaicJSON.from_features(
  items.get('features'), 
  minzoom=6, 
  maxzoom=12, 
  accessor=lambda feature: feature['assets']['browse']['href']
)

res = requests.post(
    url=f"https://{titiler_host}/mosaicjson/mosaics",
    headers={
        "Content-Type": "application/vnd.titiler.mosaicjson+json",
    },
    json=mosaicdata.dict(exclude_none=True))

assert res.status_code == 201
mosaicjson_mosaic = res.json()



In [39]:
def get_link_by_href(json: dict, rel: str):
  return next(filter(
      lambda x: x["rel"] == rel, json["links"]))["href"]

mosaicjson_mosaic = res.json()

self_href = get_link_by_href(mosaicjson_mosaic, "self")
assert res.headers['location'] == self_href

assert requests.get(get_link_by_href(mosaicjson_mosaic,
             "mosaicjson")).status_code == 200
assert requests.get(get_link_by_href(
    mosaicjson_mosaic, "tilejson")).status_code == 200

# todo: find a tile that renders in a reasonable amount of time
#tile_template = get_link_by_href(mosaicjson_mosaic, "tiles")
#tile_url = tile_template.replace("{x}", "0").replace("{y}", "0").replace("{z}", "0")
#assert requests.get(tile_url).status_code == 200

assert requests.get(get_link_by_href(mosaicjson_mosaic, "wmts")).status_code == 200

assert requests.delete(self_href).status_code == 204
